# PRASYARAT

1. Install Selenium
2. Install Web Driver yang digunakan (misal chrome install web driver chrome, misal edge instal web driver edge) - versinya sesuaikan
3. Akses Maps Menggunakan Place Id (Search Place Id Finder buat cari)
4. Inspect Element

### Import Library

In [207]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from itertools import zip_longest




### Konfigurasi Web Driver

In [188]:
# Konfigurasi WebDriver
options = webdriver.EdgeOptions()
options.use_chromium = True
# options.add_argument("headless")  # Untuk menjalankan browser dalam mode headless
driver = webdriver.Edge(options=options)
wait = WebDriverWait(driver, 5)



### Tentukan target lokasi

In [189]:
# Buka halaman Google Maps berdasarkan place id
ahass_panglima_jaya = "ChIJy8o9kNH71y0RMPCH2a8jyIE"
ahass_mutiara_motor = "ChIJM7Q_Ns371y0RgsVCneDEVvw"
ahass_dinoyo = "ChIJC1wN5bD71y0RwmNs98Ch4OE"

# driver.get(f'https://www.google.com/maps/place/?q=place_id:{ahass_panglima_jaya}')
# driver.get(f'https://www.google.com/maps/place/?q=place_id:{ahass_mutiara_motor}')
driver.get(f'https://www.google.com/maps/place/?q=place_id:{ahass_dinoyo}')

###  Karena letak komen dan review ada di sidebar, pastikan elemen tersebut bisa kita akses

In [190]:

sidebar = wait.until(EC.presence_of_element_located((By.ID, 'QA0Szd')))

### Define Sidebar Konten untuk di scrool (karena sidebar ada 2 colom, colom button, dan kolom konten)

In [191]:
# Define the element to scroll
element_to_scroll = sidebar.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div')

### Define Fungsi untuk scrool

In [192]:
# Function to scroll to the element
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(false);", element)
    
    time.sleep(5)  # Wait for 5 seconds after scrolling

### Scrool sampe button more review ditemukan

In [193]:
# Scroll until the "More reviews" button is visible
button_more_reviews = None
while not button_more_reviews:
    try:
        button_more_reviews = element_to_scroll.find_element(By.CSS_SELECTOR, 'div > button[aria-label*="ulasan lainnya" i], div > button[aria-label*="reviews" i]')
    except:
        scroll_to_element(element_to_scroll)

### Memastikan elemen button tampil dan siap di klik

In [194]:
# Scroll a bit more to ensure the button is fully visible
scroll_to_element(button_more_reviews)
time.sleep(5)

# Wait until the button is clickable
wait = WebDriverWait(driver, 5)
button_more_reviews = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div > button[aria-label*="ulasan lainnya" i], div > button[aria-label*="reviews" i]')))


### Klik Button 

In [195]:
# Click the "More reviews" button
button_more_reviews.click()

### Define Sidebar Baru (setelah kita klik tombol reviews)

In [196]:
element_to_scroll = sidebar.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf')

### Buat Fungsi baru untuk auto scrolling sampai bawah

In [197]:
# Fungsi untuk melakukan scroll ke bawah
def scroll_to_bottom(element):
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
    time.sleep(10)  # Jeda 10 detik setelah melakukan scroll


### Jalankan auto scrooling sampai limit scrool habis/ pentokin

In [198]:
# Lakukan scroll ke bawah sampai mentok
limit_scrool = 2
while True:
    previous_height = driver.execute_script("return arguments[0].scrollHeight", element_to_scroll)
    scroll_to_bottom(element_to_scroll)
    current_height = driver.execute_script("return arguments[0].scrollHeight", element_to_scroll)
    if current_height == previous_height or limit_scrool == 0:
        break
    limit_scrool -= 1

### Cari elemen html yang mau kita cari dari komen section (nama-status-year-star-comments)

In [199]:
names = driver.find_elements(By.CSS_SELECTOR, 'div.d4r55')
status = driver.find_elements(By.CSS_SELECTOR, 'div.RfnDt')
year = driver.find_elements(By.CSS_SELECTOR, 'div.DU9Pgb span.rsqaWe')

star = driver.find_elements(By.CSS_SELECTOR, 'span.kvMYJc[aria-label*="stars"]')
if (len(star) < 1) : #buat validasi bahasa
    star = driver.find_elements(By.CSS_SELECTOR, 'span.kvMYJc[aria-label*="bintang"]')

comments = driver.find_elements(By.CSS_SELECTOR, 'div.MyEned')

for st in status:
    print(len(st.text.split('·')))

2
1
1
2
2
1
1
1
2
1
1
1
2
2
2
1
1
1
2


### Check sebelum insert

In [210]:
name_list = []
reviewer_status_list = []
total_reviews_list = []
period_list = []
star_list = []
comment_list = []

print("len names", len(names))
print("len status", len(status))
print("len year", len(year))
print("len star", len(star))
print("len comments", len(comments))

len names 19
len status 19
len year 19
len star 19
len comments 14


In [211]:

for name, stat, period, st, comment in zip_longest(names, status, year, star, comments, fillvalue=""):
    print("\nName:", name.text)
    name_list.append(name.text)
    # Karena pada status bisa mereturn status reviewer dan total jumlah review, jadi kita cleaning sekalian
    if len(stat.text.split('·')) == 2:
        print("Reviewer Status :", stat.text.split('·')[0])
        print("Total Reviews :", stat.text.split('·')[1])
        reviewer_status_list.append(stat.text.split('·')[0])
        total_reviews_list.append(stat.text.split('·')[1])
    else :
        # karena gak semua status memiliki status reviewer dan total jumlah review
        print("Status:", stat.text.split('·')[0])
        print("Reviewer Status:", "Non Local Guide")
        reviewer_status_list.append("Non Local Guide")
        total_reviews_list.append(stat.text.split('·')[0])
    print("Periode:", period.text)
    period_list.append(period.text)
    try:
        # Kita split karena outputnya adalah ex "4 Bintang"
        print("Star:", st.get_attribute('aria-label').split()[0])
        star_list.append(st.get_attribute('aria-label').split()[0])
    except:
        # Kadang setelah scrool tidak tergenerate semua htmlnya (antisipasi)
        print("Star: NaN")
        star_list.append(float('nan'))
    try :
        print("Comment:", comment.text) 
        comment_list.append(comment.text)
    except : 
        print("Comment: Comment not found") 
        comment_list.append("comment not found")
    print("---")


Name: Agmarina Vica Mayelsa
Reviewer Status : Local Guide 
Total Reviews :  56 ulasan
Periode: setahun lalu
Star: 5
Comment: Kesini hari minggu ternyata buka karena sempet muter2 tapi pada tutup huhu. Mbaknya sabar nanyain kita perlu apa aja perbaikannya. Terus disediakan air untuk minum bahkan bisa buat kopi sendiri. Mas mas mekaniknya juga sabar mendengarkan … Lainnya
---

Name: Alfiah
Status: 2 ulasan
Reviewer Status: Non Local Guide
Periode: 5 bulan lalu
Star: 5
Comment: Staff dan mechanic sangat ramah.. Disediain air aqua juga bisa buat kopi.. Transparan dalam memberitahukan masalah motor dan kasih advice terbaik buat customer.. Pelayanan di AHASS DINOYO sangat memuaskan.. ❤👍 …
---

Name: Seka Riye
Status: 1 ulasan
Reviewer Status: Non Local Guide
Periode: 6 hari lalu
Star: 1
Comment: Awal cerita saya ke ahas, mengeluhkan vario 150 roda blakang goyang saat berkendara , karyawannya mau ngecek, kata karyawannya penyebabnya roda depan udah gundul dan harus di ganti, dan saya iyakan 

### Insert ke data frame biar rapih

In [212]:
df = pd.DataFrame({
    'Name': name_list,
    'Reviewer Status': reviewer_status_list,
    'Total Reviews': total_reviews_list,
    'Periode': period_list,
    'Star': star_list,
    'Comment': comment_list
})

df

,Name,Reviewer Status,Total Reviews,Periode,Star,Comment
0,Agmarina Vica Mayelsa,Local Guide,56 ulasan,setahun lalu,5,Kesini hari minggu ternyata buka karena sempet...
1,Alfiah,Non Local Guide,2 ulasan,5 bulan lalu,5,Staff dan mechanic sangat ramah.. Disediain ai...
2,Seka Riye,Non Local Guide,1 ulasan,6 hari lalu,1,"Awal cerita saya ke ahas, mengeluhkan vario 15..."
3,dini hidayah,Local Guide,78 ulasan,seminggu lalu,5,"Pelayanan cepat, antri pun ngga lama. Ganti ol..."
4,typh,Local Guide,26 ulasan,4 minggu lalu,1,"Rem belakang seret, piston d kasih stempet doa..."
5,Diki Wahyu A,Non Local Guide,6 ulasan,sebulan lalu,3,Pelayanan & mekaniknya cukup baik . Tapi sayan...
6,Candra Tungtung,Non Local Guide,5 ulasan,5 bulan lalu,5,Mekaniknya baik ramah jujur hahah.. Ga kawatir...
7,king rester,Non Local Guide,2 ulasan,4 bulan lalu,5,Mekanik nya bagus
8,ANDI,Local Guide,44 ulasan,setahun lalu,5,Pengalaman servis disini staf dan mekaniknya r...
9,M. Ismail Ahmad,Non Local Guide,12 ulasan,setahun lalu,5,Sangat rekomendasi sekali servis disini karena...
